<div class="alert alert-block alert-info">
<b>Vaja: </b> 
Uvozite dataset <i>WineQuality.csv</i>, ki vsebuje oceno kvalitete portugalskega vina "Vinho Verde".
    
Ustvarite KNeighborsRegressor model, ki bo napovedoval kvaliteto.
    
Za primerjavo - naš najboljši model je imel sledeče karakteristike:
    
    RMSE:        0.654
    KFold:       9
    k-neighbors: 13
</div>

# Uvoz podatkov

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
wine_quality = pd.read_csv("data/WineQuality.csv", header=None)

In [9]:
wine_quality.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4 %,5.0
1,2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8 %,5.0
2,3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8 %,5.0
3,4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8 %,6.0
4,5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4 %,5.0


# Cleaning data

In [10]:
wine_quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1599 non-null   int64  
 1   1       1582 non-null   float64
 2   2       1590 non-null   float64
 3   3       1579 non-null   float64
 4   4       1582 non-null   float64
 5   5       1584 non-null   float64
 6   6       1587 non-null   float64
 7   7       1586 non-null   float64
 8   8       1580 non-null   float64
 9   9       1580 non-null   float64
 10  10      1585 non-null   float64
 11  11      1581 non-null   object 
 12  12      1585 non-null   float64
dtypes: float64(11), int64(1), object(1)
memory usage: 162.5+ KB


In [11]:
wine_quality.columns = ["ID", "fixedacidity", "volatileacidity", "citricacid",
"residualsugar",
"chlorides",
"freesulfurdioxide",
"totalsulfurdioxide",
"density",
"pH",
"sulphates",
"alcohol",
"quality"]

wine_quality.head()

,ID,fixedacidity,volatileacidity,citricacid,residualsugar,chlorides,freesulfurdioxide,totalsulfurdioxide,density,pH,sulphates,alcohol,quality
0,1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4 %,5.0
1,2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8 %,5.0
2,3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8 %,5.0
3,4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8 %,6.0
4,5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4 %,5.0


In [12]:
wine_quality.info()
#alkohol je object, želimo float16

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  1599 non-null   int64  
 1   fixedacidity        1582 non-null   float64
 2   volatileacidity     1590 non-null   float64
 3   citricacid          1579 non-null   float64
 4   residualsugar       1582 non-null   float64
 5   chlorides           1584 non-null   float64
 6   freesulfurdioxide   1587 non-null   float64
 7   totalsulfurdioxide  1586 non-null   float64
 8   density             1580 non-null   float64
 9   pH                  1580 non-null   float64
 10  sulphates           1585 non-null   float64
 11  alcohol             1581 non-null   object 
 12  quality             1585 non-null   float64
dtypes: float64(11), int64(1), object(1)
memory usage: 162.5+ KB


In [13]:
wine_quality["alcohol"] = wine_quality["alcohol"].str.rstrip(" %").astype("float")/100.0

In [14]:
wine_quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  1599 non-null   int64  
 1   fixedacidity        1582 non-null   float64
 2   volatileacidity     1590 non-null   float64
 3   citricacid          1579 non-null   float64
 4   residualsugar       1582 non-null   float64
 5   chlorides           1584 non-null   float64
 6   freesulfurdioxide   1587 non-null   float64
 7   totalsulfurdioxide  1586 non-null   float64
 8   density             1580 non-null   float64
 9   pH                  1580 non-null   float64
 10  sulphates           1585 non-null   float64
 11  alcohol             1581 non-null   float64
 12  quality             1585 non-null   float64
dtypes: float64(12), int64(1)
memory usage: 162.5 KB


In [15]:
wine_quality.head()

,ID,fixedacidity,volatileacidity,citricacid,residualsugar,chlorides,freesulfurdioxide,totalsulfurdioxide,density,pH,sulphates,alcohol,quality
0,1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,0.094,5.0
1,2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,0.098,5.0
2,3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,0.098,5.0
3,4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,0.098,6.0
4,5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,0.094,5.0


In [16]:
#lahko bi pretvorili float64 v bolj optimalno obliko.

In [17]:
wine_quality = wine_quality.drop("ID", axis="columns")

wine_quality.head()

,fixedacidity,volatileacidity,citricacid,residualsugar,chlorides,freesulfurdioxide,totalsulfurdioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,0.094,5.0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,0.098,5.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,0.098,5.0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,0.098,6.0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,0.094,5.0


In [19]:
wine_quality.isnull().sum()

fixedacidity          17
volatileacidity        9
citricacid            20
residualsugar         17
chlorides             15
freesulfurdioxide     12
totalsulfurdioxide    13
density               19
pH                    19
sulphates             14
alcohol               18
quality               14
dtype: int64

In [20]:
#preverimo da ni null vrednosti

wine_quality = wine_quality.dropna(axis=0)

In [21]:
wine_quality.isnull().sum()

fixedacidity          0
volatileacidity       0
citricacid            0
residualsugar         0
chlorides             0
freesulfurdioxide     0
totalsulfurdioxide    0
density               0
pH                    0
sulphates             0
alcohol               0
quality               0
dtype: int64

# Seperating features and labels

In [18]:
#premakni stvari od zgoraj dol

# Scaling features

In [25]:
from sklearn.preprocessing import MinMaxScaler

In [28]:
scaler = MinMaxScaler()

wine_quality_scaled = pd.DataFrame(scaler.fit_transform(wine_quality), columns = wine_quality.columns)

In [30]:
wine_quality_scaled.head()

,fixedacidity,volatileacidity,citricacid,residualsugar,chlorides,freesulfurdioxide,totalsulfurdioxide,density,pH,sulphates,alcohol,quality
0,0.247788,0.397260,0.00,0.068493,0.107023,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.4
1,0.283186,0.520548,0.00,0.116438,0.143813,0.338028,0.215548,0.494126,0.362205,0.209581,0.215385,0.4
2,0.283186,0.438356,0.04,0.095890,0.133779,0.197183,0.169611,0.508811,0.409449,0.191617,0.215385,0.4
3,0.584071,0.109589,0.56,0.068493,0.105351,0.225352,0.190813,0.582232,0.330709,0.149701,0.215385,0.6
4,0.247788,0.397260,0.00,0.068493,0.107023,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.4


# Building a model

In [35]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
knn = KNeighborsRegressor(algorithm="brute", n_neighbors=13)

In [32]:
#izberemo značilke, na katerih bomo trenirali.

In [36]:
#najprej izberemo vse:
features_all = wine_quality_scaled.columns

In [37]:
features_all.tolist()

['fixedacidity',
 'volatileacidity',
 'citricacid',
 'residualsugar',
 'chlorides',
 'freesulfurdioxide',
 'totalsulfurdioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [50]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, train_test_split

In [53]:
shuffled_index = np.random.permutation(wine_quality_scaled.index)
wine_quality_scaled = wine_quality_scaled.reindex(shuffled_index)

In [54]:
split_one = wine_quality_scaled.iloc[0:700].copy()
split_two = wine_quality_scaled.iloc[700:].copy()

In [55]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

train_one = split_one
test_one = split_two
train_two = split_two
test_two = split_one

In [65]:
train_one.head()

,fixedacidity,volatileacidity,citricacid,residualsugar,chlorides,freesulfurdioxide,totalsulfurdioxide,density,pH,sulphates,alcohol,quality,predicted_quality
6,0.292035,0.328767,0.06,0.047945,0.095318,0.197183,0.187279,0.464758,0.440945,0.077844,0.153846,0.4,0.44
230,0.212389,0.585616,0.04,0.075342,0.125418,0.154930,0.215548,0.409692,0.480315,0.161677,0.153846,0.2,0.48
1171,0.398230,0.438356,0.68,0.054795,0.672241,0.239437,0.204947,0.473568,0.125984,0.598802,0.107692,0.6,0.64
466,0.513274,0.356164,0.24,0.130137,0.155518,0.394366,0.166078,0.714391,0.393701,0.203593,0.230769,0.4,0.68
61,0.327434,0.380137,0.26,0.082192,0.120401,0.140845,0.130742,0.552863,0.448819,0.119760,0.123077,0.2,0.64


In [84]:
# First half
knn.fit(train_one[['fixedacidity',
 'volatileacidity',
 'citricacid',
 'residualsugar',
 'chlorides',
 'freesulfurdioxide',
 'totalsulfurdioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']], train_one['quality'])
test_one["predicted_quality"] = knn.predict(test_one[['fixedacidity',
 'volatileacidity',
 'citricacid',
 'residualsugar',
 'chlorides',
 'freesulfurdioxide',
 'totalsulfurdioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']])
iteration_one_rmse = mean_squared_error(test_one['quality'], test_one['predicted_quality'])**(1/2)

In [85]:
iteration_one_rmse

0.1334348983336338

In [86]:
# Second half
knn.fit(train_two[['fixedacidity',
 'volatileacidity',
 'citricacid',
 'residualsugar',
 'chlorides',
 'freesulfurdioxide',
 'totalsulfurdioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']], train_two['quality'])
test_two["predicted_quality"] = knn.predict(test_two[['fixedacidity',
 'volatileacidity',
 'citricacid',
 'residualsugar',
 'chlorides',
 'freesulfurdioxide',
 'totalsulfurdioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']])
iteration_two_rmse = mean_squared_error(test_two['quality'], test_two["predicted_quality"])**(1/2)

In [87]:
iteration_two_rmse

0.13488920346151442

In [93]:
wine_quality_scaled.loc[wine_quality_scaled.index[0:159], 'fold'] = 1
wine_quality_scaled.loc[wine_quality_scaled.index[159:318], "fold"] = 2
wine_quality_scaled.loc[wine_quality_scaled.index[318:477], "fold"] = 3
wine_quality_scaled.loc[wine_quality_scaled.index[477:636], "fold"] = 4
wine_quality_scaled.loc[wine_quality_scaled.index[636:795], "fold"] = 5
wine_quality_scaled.loc[wine_quality_scaled.index[795:954], 'fold'] = 6
wine_quality_scaled.loc[wine_quality_scaled.index[954:1113], "fold"] = 7
wine_quality_scaled.loc[wine_quality_scaled.index[1113:1272], "fold"] = 8
wine_quality_scaled.loc[wine_quality_scaled.index[1272:1430], "fold"] = 9

In [94]:
#preverimo če je enakomerno:
wine_quality_scaled['fold'].value_counts()

8.0    159
7.0    159
6.0    159
5.0    159
4.0    159
3.0    159
2.0    159
1.0    159
9.0    155
Name: fold, dtype: int64

In [95]:
wine_quality_scaled['fold'].isnull().sum()

0

In [96]:
fold_ids = [1,2,3,4,5,6,7,8,9]

In [97]:
# Prva iteracija
def train_and_validate(df, folds):
    fold_rmses = []
    for fold in fold_ids:
        # Training
        model = KNeighborsRegressor()
        train = df[df['fold'] != fold].copy()
        test = df[df['fold'] == fold].copy()
        
        model.fit(train[['fixedacidity']], train['quality'])
        
        # Predicting
        labels = model.predict(test[['fixedacidity']])
        test["predicted_quality"] = labels
        rmse = mean_squared_error(test['quality'], test["predicted_quality"])**(1/2)
        fold_rmses.append(rmse)
    return fold_rmses

In [98]:
rmses = train_and_validate(wine_quality_scaled, fold_ids)

rmses

[0.17264498054729577,
 0.17647848052397305,
 0.17395148604382593,
 0.1733139805354912,
 0.1704747348439815,
 0.19734081236889423,
 0.1628982782493535,
 0.16009431182663955,
 0.16451688804378997]

In [ ]:
#Probamo s scikit learn

In [99]:
from sklearn.model_selection import cross_val_score, KFold

In [102]:
kf = KFold(9, shuffle=True, random_state=2020)

In [110]:
knn = KNeighborsRegressor(algorithm="brute", n_neighbors=13)

In [113]:
mses = cross_val_score(knn,
                       wine_quality_scaled[['fixedacidity']],
                        wine_quality_scaled['quality'],
                        scoring = 'neg_mean_squared_error',
                        cv=kf)
mses

array([-0.02966618, -0.02319378, -0.03106844, -0.02554724, -0.02801682,
       -0.02669313, -0.02700472, -0.02621077, -0.02718598])

In [114]:
rmses = np.sqrt(np.absolute(mses))
avg_rmse = np.mean(rmses)
avg_rmse

0.16472267461573273

In [115]:
#probamo z vsemi stolpci

mses = cross_val_score(knn,
                       wine_quality_scaled[['fixedacidity', 'volatileacidity', 'citricacid', 'residualsugar',
                                            'chlorides', 'freesulfurdioxide', 'totalsulfurdioxide', 'density',
                                            'pH', 'sulphates', 'alcohol']],
                        wine_quality_scaled['quality'],
                        scoring = 'neg_mean_squared_error',
                        cv=kf)
mses

array([-0.01742697, -0.0153995 , -0.02306278, -0.01705779, -0.0149038 ,
       -0.01842708, -0.01853494, -0.01779792, -0.01770055])

In [116]:
rmses = np.sqrt(np.absolute(mses))
avg_rmse = np.mean(rmses)
avg_rmse

0.13322207229470912